In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from PIL import Image

import pandas as pd
from mask_to_submission import *
import torch
from torch.utils.data import DataLoader
from preprocessing import *
from experimental_neural_nets import Conv_test, Road_data, UNet
from road_correction import process_roads, f1_loss_numpy

In [ ]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

In [ ]:
root_dir = "data/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(100, len(files))  # Load maximum 100 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

imgs,gt_imgs = rotate_train_data(imgs,gt_imgs)
imgs,gt_imgs = flip_train_data(imgs,gt_imgs)

In [ ]:
model = UNet(base_c=8,num_layers=5)

In [ ]:
num_epochs = 5

dataset = Road_data(imgs,gt_imgs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(dataloader):

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    scheduler.step()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

In [ ]:

dataset_check = Road_data(imgs, gt_imgs)
dataloader = DataLoader(dataset_check, batch_size=400, shuffle=True)

best_t = 0
best_outlier_size = 0
best_shape_size = 0
best_loss = np.inf

for batch_idx, (data, target) in enumerate(dataloader):
    print(f'batch index : {batch_idx}')
    gt = target
    images = data
    pred = model(data)

    gt_np = gt.detach().cpu().numpy()
    pred_np = pred.detach().cpu().numpy()

    for threshold in np.linspace(0.1, 0.9, 20):
        for outlier_size in np.arange(10, 100, 10):
            for shape_size in np.arange(3, 10):

                test_pred = np.array([
                    process_roads(raw_map=raw_map, threshold=threshold, outlier_size=outlier_size, shape_size=shape_size) 
                    for raw_map in pred_np
                ])
                c_loss = f1_loss_numpy(test_pred, gt_np)
            
                if c_loss < best_loss:
                    best_loss = c_loss
                    best_t = threshold
                    best_outlier_size = outlier_size
                    best_shape_size = shape_size

                    
print( best_t)

n_example = 5

# get examples from last batch
for i in range(min(n_example,len(pred_np))):
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(6, 6))
    prediction = process_roads(raw_map=pred_np[i], threshold=best_t, outlier_size=best_outlier_size, shape_size=best_shape_size)
    axes[0].imshow(prediction, cmap='gray')
    axes[0].set_title("Prediction")
    
    axes[1].imshow(pred_np[i], cmap='gray')
    axes[1].set_title("Probability Map")

    axes[2].imshow(gt[i], cmap='gray')
    axes[2].set_title("Ground Truth")

    img_np = images[i].permute(1, 2, 0).cpu().numpy() 

    axes[3].imshow(img_np)
    axes[3].set_title("Original Image")

    axes[0].axis("off")
    axes[1].axis("off")
    axes[2].axis("off")
    axes[3].axis("off")

    plt.show()

    


In [ ]:
best_shape_size = 8
best_outlier_size = 10
# best_threshold = 0.45 - 0.50

In [ ]:
import os
import re

# Define the root directory containing the test image folders
test_dir = "data/test_set_images/"

# Helper function to perform natural sorting
def natural_sort_key(s):
    # Extract numbers from the string for sorting
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# List all subdirectories (each containing one image) and sort them naturally
test_folders = sorted(
    [folder for folder in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, folder))],
    key=natural_sort_key
)

# Calculate the number of test images (based on the number of folders)
n_test = len(test_folders)
print("Loading " + str(n_test) + " images")

# Iterate over each folder, find the image, and load it
test_imgs = []
for folder in test_folders:
    folder_path = os.path.join(test_dir, folder)
    
    # List all files in the folder (assuming only one image per folder) and sort them naturally
    image_files = sorted(
        [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))],
        key=natural_sort_key
    )
    
    # Assuming there's only one image per folder, get the image file path
    if image_files:
        image_path = os.path.join(folder_path, image_files[0])
        test_imgs.append(load_image(image_path))


In [ ]:
threshold = best_t
test_imgs_np = np.array(test_imgs)
images_test= torch.from_numpy(test_imgs_np).permute(0, 3, 1, 2)
pred= model(images_test)
pred_np = pred.detach().cpu().numpy()
prediction= np.array([process_roads(raw_map=raw_map, threshold=threshold, outlier_size=best_outlier_size, shape_size=best_shape_size) for raw_map in pred_np])

def getting_sub (prediction):

    output_dir = 'predictions_opti'
    os.makedirs(output_dir, exist_ok=True)

    # Save each prediction as an image
    pred_filenames = []
    for i in range(prediction.shape[0]):
        pred_mask = (prediction[i] > threshold).astype(np.uint8)  # Example thresholding, modify as needed
        pred_image = Image.fromarray(pred_mask * 255)  # Save the mask as a black and white image
        filename = os.path.join(output_dir, f"prediction_opti{i+1:03d}.png")
        pred_image.save(filename)
        pred_filenames.append(filename)

    # Now you can call the masks_to_submission function with the generated file paths
    submission_filename = 'UNet_opti.csv'
    masks_to_submission(submission_filename, *pred_filenames)
getting_sub(prediction)